# Systemic Risk Dashboard - A Data Science Approach to Predict the Impact of Collateralization on Systemic Risk 

In [1]:
import sys
import math
import json

import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns

from bqplot import *

import networkx as nx
import visJS2jupyter.visJS_module

from ipywidgets import Dropdown, Text, HBox, Button, VBox, RadioButtons, Layout
from IPython.display import display, clear_output

from webcolors import rgb_percent_to_hex

In [2]:
# Change notebook container width to full-screen (100%)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
df = pd.read_csv('df_exposures.csv')
with open('nodes.json') as fp:
    nodes = json.load(fp)
with open('edges.json') as fp:
    edges = json.load(fp)

regulations = ['uncollateralized', 'vm_collateralized', 'im_vm_collateralized']
financial_system_ids = df['FS'].unique().tolist()

In [4]:
def load_data(seed=42):
    np.random.seed(seed)
    df = pd.read_csv('df_exposures.csv')
    with open('nodes.json') as fp:
        nodes = json.load(fp)
    with open('edges.json') as fp:
        edges = json.load(fp)

    regulations = ['uncollateralized', 'vm_collateralized', 'im_vm_collateralized']
    financial_system_ids = df['FS'].unique().tolist()
    return df, nodes, edges, regulations, financial_system_ids

class Model():
    def __init__(self, df, nodes, edges, regulations, financial_system_ids):
        self.df = df
        self.nodes = nodes
        self.edges = edges
        self.regulations = regulations
        self.financial_system_ids = financial_system_ids
        self.menu = Menu(regulations, financial_system_ids)
        self.w = None
  
    def draw_network(self):
        reg = self.menu.reg_selector.value
        fs_id = self.menu.fs_selector.value
        G_nodes = self.nodes[reg][fs_id]
        G_edges = self.edges[reg][fs_id]
        
        graph_type = self.menu.graph_type
        G = nx.Graph()
        G.add_edges_from(G_edges)
        pos = nx.spring_layout(G)
        nodes_dict = [{'id':'CP_' + str(n[0]),
                       'node_label': 'CP_' + str(n[0]),
                        'x':pos[n[0]][0]*1000,
                        'y':pos[n[0]][1]*1000,
                        'node_size':n[1]['BaselEEPE+']/100000,
                        'color': mpl.colors.rgb2hex(cm.get_cmap('bwr')(n[1]['BaselEEPE_rho+'])),
                       } for n in G_nodes] 

        node_map = dict(zip([n[0] for n in G_nodes], range(len(G_nodes)))) 
        edges_dict = [{'source':node_map[G_edges[i][0]], 
                       'target':node_map[G_edges[i][1]], 
                       'edge_width': math.log10(G_edges[i][2]['BaselEEPE']/10000 + 1),  # try taking log10()
                              } for i in range(len(G_edges))]

        if graph_type.value == 'Risk Graph':
            self.w = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                                      node_size_multiplier=0.5,
                                      node_size_transform = '',
                                      node_size_field='node_size',
                                      node_label_field='node_label',
                                      node_scaling_label_enabled=False,
                                      node_color_highlight_border='red',
                                      node_color_highlight_background='#D3918B',
                                      node_color_hover_border='blue',
                                      node_color_hover_background='#8BADD3',
                                      node_font_size=25,
                                      edge_width_field='edge_width',
                                      edge_arrow_to=True,
                                      physics_enabled=True,
                                      edge_color_highlight='#8A324E',
                                      edge_color_hover='#8BADD3',
                                      edge_width=3,
#                                       edge_width='edge_width',
                                      max_velocity=15,
                                      min_velocity=1,
                                      graph_height=430,
                                      graph_width='40%',
                                      edge_smooth_enabled = True
                                                        )

            self.w.data = self.w.data.replace("""height="1200px" width="100%""", """height="auto" width="40%""") # change size of canvas
            display(self.w)

        else:
            self.w = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                                                                physics_enabled=True,
                                                                graph_height=430,
                                                                graph_width='40%',
                                                                max_velocity=15,
                                                                min_velocity=1,
                                                             )
            display(self.w)
        
class Menu():
    def __init__(self, regulations, financial_system_ids):
        self.regulations = regulations
        self.financial_system_ids = financial_system_ids
        
        self.reg_text = Text("Regulatory Regime:")
        self.reg_selector = Dropdown(options = regulations) 
        self.fs_text = Text("Financial System:")
        self.fs_selector = Dropdown(options = financial_system_ids)
        self.graph_type = RadioButtons(options=['Trade Relations', 'Risk Graph'],layout=Layout(width='20%', height='80px'), value='Risk Graph')
        self.draw_button = Button(description="Load")
        self.is_menu_displayed = False
    
    def show(self):      
        if self.is_menu_displayed == False: 
            display(HBox([self.graph_type, VBox([self.fs_text, self.fs_selector]), VBox([self.reg_text, self.reg_selector]), self.draw_button]))
            self.is_menu_displayed = True

In [5]:
df, nodes, edges, regulations, financial_system_ids = load_data()
model = Model(df, nodes, edges, regulations, financial_system_ids)

In [6]:
# bqplot graph 1
x1 = model.regulations
df_total1 = model.df[model.regulations][(model.df['metric']=='BaselEEPE+') & (model.df['FS']==model.menu.fs_selector.value)].sum()
y1 = df_total1.as_matrix()/1000000

xs1 = OrdinalScale()
ys1 = LinearScale()

bar1 = Bars(x=x1, y=y1, 
            scales={'x': xs1, 'y': ys1}, 
            colors=['#1f77b4', '#ff7f0e', '#2ca02c'],
            padding=0.2
           )
xax1 = Axis(scale=xs1)
yax1 = Axis(scale=ys1,
            label='EEPE+ in USD (in millions)', 
            label_offset='47px', 
            orientation='vertical', 
            tick_format='0.1f', 
            grid_lines='solid'
           )
fig_layout = Layout(width='50%',
                    display='flex',
                    align_items='stretch',
                    justify_content='flex-start',
                    max_aspect_ratio=100,
                   )

fig1 = Figure(title='Impact of Collateralization on Systemic Risk', 
              marks=[bar1], 
              axes=[xax1, yax1], 
              layout=fig_layout,
              animation_duration=1000
             )

In [7]:
# bqplot graph 2

x2 = ['CP_{}'.format(x) for x in range (0, 14)]
df_eepep = model.df[['CP']+model.regulations][(model.df['metric']=='BaselEEPE+') & (model.df['FS']==model.menu.fs_selector.value)].set_index('CP')
y2 = df_eepep.T.as_matrix()/1000000

xs2 = OrdinalScale()
ys2 = LinearScale(min=0)

bar2 = Bars(x=x2, y=y2, 
            type='grouped',
            scales={'x': xs2, 'y': ys2}, 
            colors=['#1f77b4', '#ff7f0e', '#2ca02c'],
            labels=model.regulations,
            display_legend=True,
            padding=0.2)

xax2 = Axis(scale=xs2)

yax2 = Axis(scale=ys2,
            label='EEPE+ in USD (in millions)', 
            label_offset='47px', 
            orientation='vertical', 
            tick_format='0.1f', 
            grid_lines='solid')
fig2 = Figure(title='Impact of Collateralization on Systemic Risk By Counterparty', 
              marks=[bar2], 
              axes=[xax2, yax2], 
              legend_location='top-right',
              layout=fig_layout,
              animation_duration=1000
             )

In [8]:
# bqplot graph 3

x3 = ['uncollateralized', 'vm_collateralized', 'im_vm_collateralized', '     '] # hacky legend stuff, should change in final to be model.regulations perhaps

df_eepep_rhop = model.df[['CP']+model.regulations][(model.df['metric']=='BaselEEPE_rho+') & (model.df['FS']==model.menu.fs_selector.value)].set_index('CP')
df_eepep_rhop[df_eepep_rhop.columns] = df_eepep_rhop[df_eepep_rhop.columns].div(df_eepep_rhop[df_eepep_rhop.columns].sum(axis=0), axis=1).multiply(100)
df_eepep_rhop['     '] = 0 #five spaces
y3 = df_eepep_rhop.as_matrix()

xs3 = OrdinalScale()
ys3 = LinearScale(min=0)
cs3 = OrdinalColorScale(colors=['Red','Blue'])

NUM_COLORS3 = 15
CMAP3 = cm.get_cmap('tab20c')
colors3 = [CMAP3(1.*i/NUM_COLORS3) for i in range(0,NUM_COLORS3)]

colors3_hex = [rgb_percent_to_hex(((str(color[0]*100)+'%'), (str(color[1]*100)+'%'), (str(color[2]*100)+'%'))) for color in colors3]

bar3 = Bars(x=x3, y=y3, 
            scales={'x': xs3, 
                    'y': ys3,
                    'color' : cs3
                   },
            labels=['CP_{}'.format(x) for x in range(0, NUM_COLORS3)],
            colors=colors3_hex,
            display_legend=True,
            padding=0.2
           )

xax3 = Axis(scale=xs3)
yax3 = Axis(scale=ys3,
            label='Rho+ In %', 
            label_offset='49px', 
            orientation='vertical', 
            tick_format='0.1f', 
            grid_lines='solid'
           )

fig3 = Figure(title='Concentration of Risk', 
              marks=[bar3], 
              axes=[xax3, yax3], 
              legend_location='top-right',
              layout=fig_layout,
              animation_duration=1000, 
             )

In [9]:
figsw1_list = [Figure(layout=Layout(width='50%')), fig1]
display(widgets.HBox(figsw1_list, align_content = 'stretch'))

HBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(width='50%'), scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Figure(animation_duration=1000, axes=[Axis(scale=OrdinalScale()), Axis(label='EEPE+ in USD (in millions)', label_offset='47px', orientation='vertical', scale=LinearScale(), tick_format='0.1f')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(align_items='stretch', display='flex', justify_content='flex-start', width='50%'), marks=[Bars(colors=['#1f77b4', '#ff7f0e', '#2ca02c'], interactions={'hover': 'tooltip'}, padding=0.2, scales={'x': OrdinalScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array(['uncollateralized', 'vm_collateralized', 'im_vm_collateralized'],
      dtype='<U20'), y=array([110.1633856 ,  40.55716875,   9.18052591]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Impact of Collateralization on Systemic Risk')))

In [10]:
figs23_list = [fig2, fig3]
display(widgets.HBox(figs23_list, align_content = 'stretch'))

HBox(children=(Figure(animation_duration=1000, axes=[Axis(scale=OrdinalScale()), Axis(label='EEPE+ in USD (in millions)', label_offset='47px', orientation='vertical', scale=LinearScale(min=0.0), tick_format='0.1f')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(align_items='stretch', display='flex', justify_content='flex-start', width='50%'), marks=[Bars(colors=['#1f77b4', '#ff7f0e', '#2ca02c'], display_legend=True, interactions={'hover': 'tooltip'}, labels=['uncollateralized', 'vm_collateralized', 'im_vm_collateralized'], padding=0.2, scales={'x': OrdinalScale(), 'y': LinearScale(min=0.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, type='grouped', x=array(['CP_0', 'CP_1', 'CP_2', 'CP_3', 'CP_4', 'CP_5', 'CP_6', 'CP_7',
       'CP_8', 'CP_9', 'CP_10', 'CP_11', 'CP_12', 'CP_13'], dtype='<U5'), y=array([[16.41184463,  7.34325207, 13.33568375, 10.07590049,  7.25830559,
        13.59369258,  3.49313757,  5.92010757,  6.39730388,  8.30187075,
         4.96875025,  7.44240696,  5.62112951],
       [ 3.20079473,  1.39372843,  3.75794329,  4.04858279,  3.38871265,
         3.23666843,  1.92709781,  3.60873123,  1.15313725,  3.05867049,
         3.73631094,  2.78815713,  5.25863358],
       [ 0.57951074,  0.16646472,  1.04307846,  0.26002273,  1.85326487,
         0.77958458,  0.0545572 ,  0.29932582,  0.04137265,  1.12910997,
         0.94402034,  0.47892422,  1.55128961]]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Impact of Collateralization on Systemic Risk By Counterparty'), Figure(animation_duration=1000, axes=[Axis(scale=OrdinalScale()), Axis(label='Rho+ In %', label_offset='49px', orientation='vertical', scale=LinearScale(min=0.0), tick_format='0.1f')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(align_items='stretch', display='flex', justify_content='flex-start', width='50%'), marks=[Bars(colors=['#3182bd', '#6baed6', '#9ecae1', '#e6550d', '#fd8d3c', '#fdae6b', '#31a354', '#74c476', '#a1d99b', '#756bb1', '#9e9ac8', '#bcbddc', '#636363', '#969696', '#bdbdbd'], display_legend=True, interactions={'hover': 'tooltip'}, labels=['CP_0', 'CP_1', 'CP_2', 'CP_3', 'CP_4', 'CP_5', 'CP_6', 'CP_7', 'CP_8', 'CP_9', 'CP_10', 'CP_11', 'CP_12', 'CP_13', 'CP_14'], padding=0.2, scales={'x': OrdinalScale(), 'y': LinearScale(min=0.0), 'color': OrdinalColorScale(colors=['Red', 'Blue'])}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array(['uncollateralized', 'vm_collateralized', 'im_vm_collateralized',
       '     '], dtype='<U20'), y=array([[14.8977308 ,  7.89205664,  6.31239153,  0.        ],
       [ 6.66578286,  3.43645396,  1.8132373 ,  0.        ],
       [12.10536847,  9.26579297, 11.36185955,  0.        ],
       [ 9.14632428,  9.98240981,  2.83232935,  0.        ],
       [ 6.58867332,  8.35539747, 20.186914  ,  0.        ],
       [12.33957408,  7.98050882,  8.49172027,  0.        ],
       [ 3.17086984,  4.75155902,  0.59427097,  0.        ],
       [ 5.37393394,  8.89788745,  3.26044306,  0.        ],
       [ 5.80710537,  2.84323903,  0.45065664,  0.        ],
       [ 7.53596188,  7.54162725, 12.29896828,  0.        ],
       [ 4.51034636,  9.21245505, 10.28285688,  0.        ],
       [ 6.75578997,  6.87463454,  5.21674057,  0.        ],
       [ 5.10253881, 12.96597801, 16.89761159,  0.        ]]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Concentration of Risk')))

In [11]:
clear_output(wait=True)
model.menu.show()

HBox(children=(RadioButtons(index=1, layout=Layout(height='80px', width='20%'), options=('Trade Relations', 'Risk Graph'), value='Risk Graph'), VBox(children=(Text(value='Financial System:'), Dropdown(options=(0, 1, 2, 3, 4), value=0))), VBox(children=(Text(value='Regulatory Regime:'), Dropdown(options=('uncollateralized', 'vm_collateralized', 'im_vm_collateralized'), value='uncollateralized'))), Button(description='Load', style=ButtonStyle())))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [114]:
from IPython.display import Javascript
from ipywidgets import widgets

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
model.menu.draw_button.on_click(run_all)

In [115]:
### ALL CODE BELOW WILL BE EXECUTED BY 'LOAD' BUTTON ###

In [116]:
model.draw_network()

In [117]:
# bqplot graph 1 update:
df_total1 = model.df[model.regulations][(model.df['metric']=='BaselEEPE+') & (model.df['FS']==model.menu.fs_selector.value)].sum()
y1 = df_total1.as_matrix()/1000000
bar1.y = y1

In [118]:
# bqplot graph 2 update:
df_eepep = model.df[['CP']+model.regulations][(model.df['metric']=='BaselEEPE+') & (model.df['FS']==model.menu.fs_selector.value)].set_index('CP')
y2 = df_eepep.T.as_matrix()/1000000
bar2.y = y2

In [119]:
# bqplot graph 3 update:
df_eepep_rhop = model.df[['CP']+model.regulations][(model.df['metric']=='BaselEEPE_rho+') & (model.df['FS']==model.menu.fs_selector.value)].set_index('CP')
df_eepep_rhop[df_eepep_rhop.columns] = df_eepep_rhop[df_eepep_rhop.columns].div(df_eepep_rhop[df_eepep_rhop.columns].sum(axis=0), axis=1).multiply(100)

# hacky adding empty column on right side for legend
df_eepep_rhop['     '] = 0
y3 = df_eepep_rhop.as_matrix()

bar3.y = y3